## Learning PuLP

PuLP es una librería de Python utilizada para modelar y resolver problemas de programación lineal y programación mixta

In [1]:
from pulp import *
from typing import Dict

Es posible especificar un nombre para el problema de optimización y si consiste en minimizar/maximizar la función objetivo

In [2]:
problem = LpProblem ("problem-name", LpMinimize)
# or LpMaximize

Para definir una variable se tienen los siguientes parámetros: 
- `name`: nombre de la variable
- `lowBound`: cota inferior
- `upBound`: cota superior
- `cat`:  categoría de la variable: `Integer` (variable entera), `Binary` (variable binaria), `Continuous` (variable continua)

In [3]:
x = LpVariable(
  name='1.1',
  lowBound=None,
  upBound=10,
  cat=const.LpInteger,
)

El estado de la solución se puede mostrar con `LpStatus[problem.status]`

In [ ]:
print("Status:", LpStatus[problem.status])

Los posibles valores que puede devolver son: 
- `Not Solved` __description__
- `Optimal` __description__
- `Infeasible` __description__ 
- `Unbounded` __description__
- `Undefined` __description__

Si se quisiera mostrar cada variable con su posible valor óptimo se tiene:

In [5]:
for v in problem.variables():
  print(v.name, "=", v.varValue)

El valor óptimo de la función objetivo se puede mostrar de la siguiente forma:

In [ ]:
print("Total Cost of Ingredients per can = ", value(problem.objective))

### Ejemplo de un Problema de Mezcla 

Una fábrica produce aceite mezclando aceites refinados, dos de origen vegetal y tres de origen no vegetal. En un mes sólo es posible refinar 200 toneladas de vegetal y 250 de no vegetal. El aceite resultante debe cumplir un valor de dureza comprendido entre 3 y 6. El costo de una tonelada para cada aceite refinado junto con su dureza aparecen en la siguiente tabla:

|        | $Veg_{1}$ | $Veg_{2}$ | $NoVeg_{1}$ | $NoVeg_{2}$ | $NoVeg_{3}$ |
| ------ | --------- | --------- | ----------- | ----------- | ----------- |
| costo  | 110       | 120       | 130         | 110         | 115         |
| dureza | 8.8       | 6.1       | 2.0         | 4.2         | 5.0         |

Se trata de refinar las cantidades apropiadas de cada aceite a fin de maximizar el beneficio de la producción final sabiendo que una tonelada del aceite producido se vende a 150

**Variables de decisión**
- $x_{1}:$ cantidad de aceite refinado $Veg_{1}$
- $x_{2}:$ cantidad de aceite refinado $Veg_{2}$
- $x_{3}:$ cantidad de aceite refinado $NoVeg_{1}$
- $x_{4}:$ cantidad de aceite refinado $NoVeg_{2}$
- $x_{5}:$ cantidad de aceite refinado $NoVeg_{3}$
- $y:$ cantidad de aceite a producir

**Restricciones**
$$\begin{matrix}
x_{1}+x_{2} \leq 200 & (\text{aceite no vegetal refinado} \leq \text{capacidad de refino vegetal}) \\
x_{3}+x_{4}+x_{5} \leq 250 & (\text{aceite no vegetal refinado} \leq \text{capacidad de refino no vegetal})  \\
8.8x_{1}+6.1x_{2}+2x_{3}+4.2x_{4}+5x_{5} \leq 6y & (\text{límite superior de dureza del aceite producido}) \\
8.8x_{1}+6.1x_{2}+2x_{3}+4.2x_{4}+5x_{5} \geq 3y & (\text{límite inferior de dureza del aceite producido}) \\
x_{1}+x_{2}+x_{3}+x_{4}+x_{5}=y & \text{suma de las cantidades de los aceites refinados = cantidad de aceite producido}\\
x_{1},x_{2},x_{3},x_{4},x_{5},x_{6},y \geq 0 & (\text{no negatividad})
\end{matrix}$$

**Función objetivo**
$$\max z = 150y - 110x_{1} - 120x_{2} - 130x_{3} - 110x_{4} - 115x_{5}$$
Esta función viene del análisis: valor del aceite producido - coste de los aceites refinados


In [ ]:
problem = LpProblem("Mezcla", LpMaximize)

veg_materials = ["Veg_1","Veg_2"]
no_veg_materials = ["No-Veg_1","No-Veg_2","No-Veg_3"]
materials = veg_materials + no_veg_materials
materials_vars:Dict[str,LpVariable] = LpVariable.dicts("Materials",materials,0,cat=const.LpInteger)
total = lpSum( [materials_vars[i] for i in materials] )


In [33]:
costs = {
  "Veg_1" : 110,
  "Veg_2" : 120,
  "No-Veg_1" : 130,
  "No-Veg_2" : 110,
  "No-Veg_3" : 115
}
viscosities = {
  "Veg_1" : 8.8,
  "Veg_2" : 6.1,
  "No-Veg_1" : 2.0,
  "No-Veg_2" : 4.2,
  "No-Veg_3" : 5.0
} 

In [34]:
problem += (
  150*total - lpSum( [costs[i]*materials_vars[i] for i in materials] ),
  "Objetive Function"
)

In [35]:
problem += (
  lpSum([materials_vars[i] for i in veg_materials]) <= 200, 
  "Constract 1" )
problem += (
  lpSum([materials_vars[i] for i in no_veg_materials]) <= 250, 
  "Constract 2" )
problem += (
  lpSum([materials_vars[i]*viscosities[i] for i in materials]) <= 6*total, 
  "Constract 3" )
problem += (
  lpSum([materials_vars[i]*viscosities[i] for i in materials]) >= 3*total, 
  "Constract 4" )

In [ ]:
problem.solve()

In [ ]:
LpStatus[problem.status]

In [ ]:
print("Total Cost of Ingredients per can = ", value(problem.objective))

for v in problem.variables():
  print(v.name, "=", v.varValue)

In [ ]:
# TODO: Visualización gráfica del problema